First, we'll import the packages we need and set some variables

In [1]:
# Import Packages
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import folium
import requests

In [2]:
api_key = 'GQLRWSCYD3BBX2KWJEBPYSYZBAGDLTKD5G1A5KBRNUOG4UKU' #  API Key removed for Github copy.
api_secret = '04TK43WE3BKYIE1BPMNUYLDWRTU24AZ3MPN0EOOWFTLDX13T' #  API Key removed for Github copy.
version = '20210101'
limit = 50
radius = 3000
city = 'Seattle, WA'

Next, create and set our map.

In [3]:
geolocator = Nominatim(user_agent='seattle')
location = geolocator.geocode(city)

lat = location.latitude + .005  # '+ .005' is to move the map NE a bit for our purposes.
lng = location.longitude + .005

map_seattle = folium.Map(location=[lat,lng], zoom_start=13)

We then make our API call to Foursquare to identify the categories of restaurants we wish to use

In [4]:
restaurants = []
url = 'https://api.foursquare.com/v2/venues/search?ll={},{}&client_id={}&client_secret={}&v={}&limit={}'.format(
        lat,
        lng,
        api_key,
        api_secret,
        version,
        limit)
response = requests.get(url).json()['response']['venues']

categories = requests.get('https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(
                            api_key,
                            api_secret,
                            version)).json()['response']['categories']

restaurant_type_ids = ['4bf58dd8d48988d1c0941735', '4bf58dd8d48988d115941735', '4f04af1f2fb6e1c99f3db0bb']

For our purposes, we will use the following categories:

    Mediterranean Restaurant 4bf58dd8d48988d1c0941735
    Middle Eastern Restaurant 4bf58dd8d48988d115941735
    Turkish Restaurant 4f04af1f2fb6e1c99f3db0bb

Next we will use futher API calls to search for each category and create a list of restaurants

In [5]:
restaurants = []
res_data = []
for type_id in restaurant_type_ids:
    url = 'https://api.foursquare.com/v2/venues/search?ll={},{}&client_id={}&client_secret={}&categoryId={}&v={}&limit={}&radius={}'.format(
            lat, lng, api_key, api_secret, type_id, version, limit, radius)
    
    response = requests.get(url).json()['response']['venues']
    for restaurant in response:
        restaurants.append(restaurant)
for restaurant in restaurants:
    res_data.append([restaurant['location']['lat'], restaurant['location']['lng'], restaurant['name']])

Our list of restaurants is named res_data, and will contain the location, name, cluster, and #ofReviews for each restaurant. The cluster and reviews will be added later. Having our list of restaurants along with their location, we will now sort them into clusters and add the cluster id to each restaurant in the list

In [6]:
coords = []
for x, y, res in res_data:
    coords.append([x, y])

k_means = KMeans(init="k-means++", n_clusters=6, n_init=20)
k_means.fit(coords)
k_means_cluster_centers = k_means.cluster_centers_
k_means_labels = k_means.labels_

colors = {'0' : 'green', '1' : 'red', '2' : 'blue', '3' : 'orange', '4' : 'purple', '5' : 'black'}
for i, cluster in enumerate(k_means_labels): #  asign a color to each restaurant based on cluster
    res_data[i].append(cluster)

We will next scrape the reviews for each of these restaurants from Yelp using Selenium. The function get_reviews() will scrape yelp and return the # of reviews for a restaurant

In [7]:
def get_reviews(name):
    
    browser.get('https://www.yelp.com/search?find_desc={}&find_loc=Downtown%2C+Seattle%2C+WA'.format(name.replace(' ', '+')))
    
    for b in browser.find_elements_by_tag_name('div[class^=\' mainAttributes\']'):
        if b.text[0:2] == '1.':
            return int(b.text.split('\n')[1])

# starts our Selenium Firefox browser
options = Options()
options.headless = True
browser = webdriver.Firefox(options=options)

# visits the website for each restaurant and scrapes the # of reviews.
for restaurant in res_data:
    name = restaurant[2]
    reviews = get_reviews(name)
    try:
        if reviews > 0:
            restaurant.append(reviews)
    except:# In the event the function returns 'None', representing no reviews, we manually enter the # 0
        restaurant.append(0)

Sample of our completed list res_data: (longitude, latitude, name, color, #ofReviews) 

In [8]:
for restaurant in res_data[0:5]:
    print(restaurant)

[47.613642, -122.32106, 'Garlic Crush', 2, 70]
[47.624748, -122.357256, 'Mint & Olive', 0, 375]
[47.604019, -122.3315, 'Gyro Express', 3, 19]
[47.604563, -122.33351, 'Mado Oven', 3, 0]
[47.629931, -122.342465, 'Gyro & More', 4, 249]


We will then figure out which cluster has the highest average # of reviews

In [9]:
averages = [0] * 6  #  each list entry here represents a cluster #
counts = [0] * 6  # records the number of restaurants in each cluster

for restaurant in res_data:
    averages[restaurant[3]] += restaurant[4]
    counts[restaurant[3]] += 1
for i, cluster in enumerate(averages):
    averages[i] = int(cluster / counts[i])
averages

[292, 401, 294, 142, 120, 181]

In this case, cluster [1] has the highest # of reviews relative to the number of restaurants in the area. This is the downtown area.

Finally, we will add our points to the map and render it

In [10]:
for i, data in enumerate(res_data):
    label = folium.Popup('{}: {} Reviews'.format(data[2], data[4]), parse_html=True)
    folium.CircleMarker([data[0], data[1]], radius=5, popup=label, 
                        color=colors['{}'.format(data[3])]).add_to(map_seattle)

In [11]:
map_seattle